# ２つの生物種のTTN遺伝子のイントロン同士の配列相同性をBLASTで評価する

**目的**
イントロン同士の配列を比較して、相同性を確認する<br>
相同性とスプライシングパターンの関連がないかを検証

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
import matplotlib.pyplot as plt
import pandas as pd

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

import subprocess
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 準備

In [ ]:
# 生物種の指定
specie1 = "human"
specie2 = "mouse"

# ディレクトリのリセット
cmd1 = "rm -rf ../data/blast"
cmd2 = f"mkdir -p ../data/blast/db/{specie1}/ ../data/blast/fasta/{specie1}/ ../data/blast/fasta/{specie2}/ ../result/blast/{specie1}_{specie2}/"
subprocess.call(cmd1.split())
subprocess.call(cmd2.split())

# クラスのインスタンス化
s1 = Seq_count()
s2 = Seq_count()

# gbkファイルの読み込み
s1.read_gbk(f"../data/gbk/{specie1}_ttn.gb")
s2.read_gbk(f"../data/gbk/{specie2}_ttn.gb")

# 遺伝子全長配列
seq1 = s1.gDNA_seq()
seq2 = s2.gDNA_seq()

## BLASTの設定

In [ ]:
# specie1のすべてのイントロン配列をfasta形式で保存
with open(f"../data/blast/fasta/{specie1}/ttn_intron.fasta","w") as f:
    for n,i in enumerate(s1.intron_list()):
        f.write(f">{specie1}_intron{str(n+1).zfill(3)}\n")
        f.write("".join(seq1[i[0]:i[1]]))
        f.write("\n")

# specie2すべてのイントロン配列を個別にfasta形式で保存
for n,i in enumerate(s2.intron_list()):
    with open(f"../data/blast/fasta/{specie2}/ttn_intron{str(n+1).zfill(3)}.fasta","w") as f:
        f.write(f">{specie2}_intron{str(n+1).zfill(3)}\n")
        f.write("".join(seq2[i[0]:i[1]]))

# BLASTデータベースの作成
cmd = f"makeblastdb -in ../data/blast/fasta/{specie1}/ttn_intron.fasta -dbtype nucl -parse_seqids -out ../data/blast/db/{specie1}/ttn_intron"
subprocess.run(cmd.split())

## BLASTの実行

In [ ]:
# すべてのイントロンに対してBLASTを実行
for i in range(s2.intron_num()):
    fmt = 6
    cmd = f"blastn -query ../data/blast/fasta/{specie2}/ttn_intron{str(i+1).zfill(3)}.fasta -db ../data/blast/db/{specie1}/ttn_intron -out ../result/blast/{specie1}_{specie2}/ttn_intron{str(i+1).zfill(3)}.txt -outfmt {fmt}"
    subprocess.run(cmd.split())